In [2]:
# Install requirements
!pip install fastapi
!pip install python-multipart
!pip install uvicorn
!pip install nest-asyncio
!pip install pyngrok
!pip install pymongo
!pip install pymongo[srv]
!pip install bcrypt
!pip install python-jose
!pip install passlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.0 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 80 kB 9.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=800784a16e4e53dfdfd0740b28185e8d15578e914bf757c195547db17bcc2ca7
  Stored in directory: /root/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built python-multipart
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 58 kB 6.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in 

In [4]:
from fastapi import FastAPI, HTTPException, Depends, Request,status, File, UploadFile
from typing import List
from hashing import Hash
from jwttoken import create_access_token
from oauth import get_current_user, get_current_active_user
from models import User
from fastapi.security import OAuth2PasswordRequestForm
from fastapi.middleware.cors import CORSMiddleware
from config import db
import uvicorn


#from PIL import Image
from pyngrok import ngrok
import nest_asyncio

In [5]:
# for dohoon's code

import os
import imutils
import numpy as np
import cv2
from PIL import Image
import matplotlib.patches as patches
import skimage
from skimage import io, transform


In [6]:
import torchvision.models as models
import torch
import torch.nn as nn
from torchvision import transforms

import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img 


import collections
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

In [7]:
#cropping

def video_partition():
    count = []
    for i in range(1, 100):
        filepath = '/content/%d.mp4' % i
        if os.path.isfile(filepath):
            video = cv2.VideoCapture(filepath)

            if not video.isOpened():
                print("Could not Open :", filepath)
                exit(0)

            length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
            width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = video.get(cv2.CAP_PROP_FPS)

            cnt = 0

            while(video.isOpened()):
                if (cnt + 1) * (int)(fps) > length:
                    break 
                ret, image = video.read()
                if(int(video.get(1)) % (int)(fps) == 0):
                    createFolder("/content/video%d" % i)
                    cv2.imwrite("/content/video%d/frame%d.png" % (i, cnt), image)

                    cnt += 1
            video.release()
        else:
            break
        count.append(cnt)

    return count, i - 1
    
def load_image(image_path):
    img = skimage.img_as_float(io.imread(image_path))
    if len(img.shape) == 2:
        img = np.array([img, img, img]).swapaxes(0, 2)
    return img

def rescale(img, input_height, input_width):
    aspect = img.shape[1] / float(img.shape[0])
    if (aspect > 1):
    # landscape orientation - wide image
        res = int(aspect * input_height)
        imgScaled = transform.resize(img, (input_width, res))
    if (aspect < 1):
        # portrait orientation - tall image
        res = int(input_width / aspect)
        imgScaled = transform.resize(img, (res, input_height))
    if (aspect == 1):
        imgScaled = transform.resize(img, (input_width, input_height))
    return imgScaled
  
def normalize(img, mean=128, std=128):
    img = (img * 256 - mean) / std
    return img

def crop_center(img, cropx, cropy):
    y, x, c = img.shape
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)
    return img[starty:starty + cropy, startx:startx + cropx]

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error:Creating directory.' + directory)
        
def prepare(img_uri):
        img = load_image(img_uri)
        img = rescale(img, 300, 300)
        img = transform.resize(img, (300, 300))
        img = normalize(img)
        return img

In [8]:
def ssd(target):
    k, v = video_partition()
    import torch
    device = torch.device('cpu')

    ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd')
    utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')


    ssd_model.to('cuda')
    ssd_model.eval()

    for j in range(1, v + 1):   
        count = 1    
        uris = list()
        for i in range(k[j - 1]):
            uris.append('/content/video%d/frame%d.png' % (j, i))

        inputs = [prepare(uri) for uri in uris]
        tensor = utils.prepare_tensor(inputs)

        with torch.no_grad():
            detections_batch = ssd_model(tensor)

        results_per_input = utils.decode_results(detections_batch)
        best_results_per_input = [utils.pick_best(results, 0.70) for results in results_per_input]
        classes_to_labels = utils.get_coco_object_dictionary()
        createFolder("/content/result%d" % j)

        i = 1

        for item in target:
          t = cv2.imread(f'/content/{item}')
          cv2.imwrite(f'/content/result{j}/{item}', t)
          i = i + 1

        for image_idx in range(len(best_results_per_input)):
            tmp = plt.imread('/content/video%d/frame%d.png' % (j, image_idx))
            image = inputs[image_idx] / 2 + 0.5
            bboxes, classes, confidences = best_results_per_input[image_idx]
            for idx in range(len(bboxes)):
                left, bot, right, top = bboxes[idx]
                x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
                if classes_to_labels[classes[idx] - 1] == 'person':         
                    tmp = rescale(tmp, 300, 300)
                    tmp = transform.resize(tmp, (300, 300))
                    if y < 0:
                        y = 0
                    if x < 0:
                        x = 0
                    cropped = tmp[int(y):int(y+h), int(x):int(x+w)]
                    plt.imsave("/content/result%d/%d.png" % (j, count), cropped)
                    count += 1

In [9]:
def GetDLModel():
    # 모델 불러와서 일부 키 값 resnet50 model에 맞게 수정
    # 아래 torch.load에서 모델을 저장한 경로만 바뀌면 됩니다.
    device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

    checkpoint2 = torch.load(
        '/content/resnet50_market_xent.pth.tar', map_location=device, encoding='latin1')
    checkpoint2['state_dict']['fc.weight'] = checkpoint2['state_dict'].pop(
        'classifier.weight')
    checkpoint2['state_dict']['fc.bias'] = checkpoint2['state_dict'].pop(
        'classifier.bias')

    # 기존의 resnet50을 market1501을 학습 시킨 모델과 출력층이 동일하게 구성
    res2 = models.resnet50()
    x = res2.fc.weight
    x = torch.narrow(x, 0, 0, 751)
    y = res2.fc.bias
    y = torch.narrow(y, 0, 0, 751)

    res2.fc.weight = nn.Parameter(x)
    res2.fc.bias = nn.Parameter(y)
    res2.fc.out_features = 751

    # 모델 덮어 씌우기
    res2.load_state_dict(checkpoint2['state_dict'])
    res2.to(device)  # 가중치를 GPU 계산과 CPU 계산 중 하나로 통일

    return res2

def getImages():
    # 이미지 폴더에서 이미지 셋 리스트에 저장

    path = '/content/result1'
    os.chdir(path)

    images = []

    # 이미지 파일 리스트에 삽입
    with os.scandir(path) as files:
        for file in files:
            if file.name.endswith('.png'):
                images.append(file.name)
    return images

class FeatureExtractor(nn.Module):
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
        self.conv1 = model.conv1
        self.bn1 = model.bn1
        self.relu = model.relu
        self.maxpool = model.maxpool
        self.layer1 = model.layer1
        self.layer2 = model.layer2
        self.layer3 = model.layer3
        self.layer4 = model.layer4
        self.global_avgpool = model.avgpool
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        return

    def featuremaps(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def forward(self, x):
        f = self.featuremaps(x)
        v = self.global_avgpool(f)
        v = v.view(v.size(0), -1)
        v = self.pool(v)
        v = self.pool(v)
        v = self.pool(v)
        v = self.pool(v)
        return v

# 이미지 특징 추출 함수

def extracting_feature(model, images, target_name, transform):
    device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
    features_ = []
    extract_feature = FeatureExtractor(model)
    for i in range(len(images)):
        if i == len(images):
            break

        path = os.path.join('/content/result1', images[i])
        img = load_img(path, target_size=(224, 224))
        img = np.array(img)
        img = transform(img)
        img = img.reshape(1, 3, 224, 224)
        img = img.to(device)
        with torch.no_grad():
            feature = extract_feature(img)
        features_.append(feature.cpu().detach().numpy().reshape(-1))
    return features_


def clusters(feat, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', n_init=5, max_iter=300)
    kmeans.fit(feat)
    result = kmeans.labels_
    labels = np.unique(result)
    return result, labels

# 이미지 transform 구성
def get_transform():
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return transform


def play(n_cluster=3, image_name=['target.png']):
    target_name = image_name
    MyResnet = GetDLModel()
    zepeto = getImages()
    trans = get_transform()
    features = extracting_feature(MyResnet, zepeto, target_name, trans)
    features = np.array(features)
    (res, labels) = clusters(features, n_cluster)

    groups = {}
    for file, cluster in zip(zepeto, res):
        if cluster not in groups.keys():
            groups[cluster] = []
            groups[cluster].append(file)
        else:
            groups[cluster].append(file)
    count = 0
    for i in range(len(labels)):
        flag = 0
        for target in target_name:
            if target in groups[i]:
                flag = 1
        if flag == 0:
            count += len(groups[i])
    return count

In [10]:
app = FastAPI()
origins = [
    "http://localhost",
    "http://localhost:3000",
    "http://localhost:8080",
]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [11]:
@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.post("/files/")
async def create_files(current_user:User = Depends(get_current_user),files: List[bytes] = File(...)):
    return {"file_sizes": [len(file) for file in files]}


@app.post("/uploadfiles")
async def create_upload_files(num : int, current_user:User = Depends(get_current_user),  files: List[UploadFile] = File(...) ):
    UPLOAD_DIRECTORY = "./"
    realname = current_user.username

    if num != current_user.target_num:
        db["users"].update_one({'username': realname}, {'$set': {'target_num': num}})

    for file in files:
        contents = await file.read()
        with open(os.path.join(UPLOAD_DIRECTORY, file.filename), "wb") as fp:
            fp.write(contents)

    for file in files:
        if(file.filename[-4:] == ".mp4"):
            if(current_user.filename == "video.mp4"):
                db["users"].update_one({'username': realname}, {'$set': {'filename': file.filename}})

        elif(file.filename[-4:] == ".png"):
            if(current_user.targetname == "default.png"):
                db["users"].update_one({'username': realname}, {'$set': {'targetname': file.filename}})

    return {"filenames": [file.filename for file in files]}

@app.post('/register')
async def create_user(request:User):
	hashed_pass = Hash.bcrypt(request.password)
	user_object = dict(request)
	user_object["password"] = hashed_pass
	user_id = db["users"].insert_one(user_object)
	return {"register":"completed"}

@app.post('/login')
async def login(request:OAuth2PasswordRequestForm = Depends()):
	user = db["users"].find_one({"username":request.username})
	if not user:
		raise HTTPException(status_code=status.HTTP_404_NOT_FOUND,detail = f'No user found with this {request.username} username')
	if not Hash.verify(user["password"],request.password):
		raise HTTPException(status_code=status.HTTP_404_NOT_FOUND,detail = f'Wrong Username or password')
	access_token = create_access_token(data={"sub": user["username"] })
	return {"access_token": access_token, "token_type": "bearer"}

@app.put("/play")
async def algo(current_user:User = Depends(get_current_active_user)):
#algorithm

    list_name = []
    list_name.append(current_user.targetname)
    numb = current_user.target_num

    ssd(list_name)
    cnt = play(numb,list_name)
    print(cnt)
    score = 0
    if cnt < 20:
        score = 20
    elif cnt < 50:
        score = 50
    elif cnt < 80:
        score = 80
    elif cnt < 100:
        score = 100
    elif cnt < 150:
        score = 150
    else:
        score = 200
#update
    realname = current_user.username
    user_score = current_user.score + score
    db["users"].update_one({'username': realname}, {'$set': {'score': user_score}})

    return {cnt : score}

In [12]:
auth_token = "29vVaP7lvt3oLllnrWLYmOVJrUn_5x7Rdz1f4HkEUkLJm2t15" #@param {type:"string"}
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel

# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar and copy the API key
os.system(f"ngrok authtoken {auth_token}")

0

In [13]:
# Create tunnel
public_url = ngrok.connect(8000, port='8000', bind_tls=True)
# Check if it exists
!ps aux | grep ngrok

root         346 18.0  0.1 724156 21460 ?        Sl   07:19   0:00 /usr/local/lib/python3.7/dist-packages/pyngrok/bin/ngrok start --none --log=stdout
root         356  0.0  0.0  39200  6668 ?        S    07:19   0:00 /bin/bash -c ps aux | grep ngrok
root         358  0.0  0.0  38572  5508 ?        S    07:19   0:00 grep ngrok


In [14]:
# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()

# Run the FastAPI app using uvicorn
print(public_url)
uvicorn.run(app)

NgrokTunnel: "https://5537-34-136-174-10.ngrok.io" -> "http://localhost:8000"


INFO:     Started server process [72]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     211.108.54.19:0 - "GET / HTTP/1.1" 200 OK
INFO:     211.108.54.19:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     211.108.54.19:0 - "GET / HTTP/1.1" 200 OK
INFO:     211.108.54.19:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     211.108.54.19:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     211.108.54.19:0 - "POST /register HTTP/1.1" 200 OK
INFO:     211.108.54.19:0 - "POST /register HTTP/1.1" 200 OK
INFO:     211.108.54.19:0 - "POST /login HTTP/1.1" 200 OK
INFO:     211.108.54.19:0 - "POST /uploadfiles?num=3 HTTP/1.1" 200 OK


Downloading: "https://github.com/NVIDIA/DeepLearningExamples/archive/torchhub.zip" to /root/.cache/torch/hub/torchhub.zip
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:14: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:18: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


83
INFO:     211.108.54.19:0 - "PUT /play HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [72]


In [ ]:
# Kill tunnel
#ngrok.kill()

ngrok.disconnect(public_url=public_url)